In [717]:
# Project 1 -RSA & Digital signature
# Proferssor: Dr.Zhong-Hui Duan
# Student: Om Patel
# Email: op20@uakron.edu

In [718]:
# all of the libraries that are needed
import random
import math
import hashlib

In [719]:
# use the iterative method to make calculations faster
def modularExponentiation(base, exponent, modulus):
    result = 1
    base = base % modulus
    # runs until the exponent is 0
    while (exponent > 0):
        # if the exponent is odd, multiply the base with the result
        if ((exponent % 2) == 1):
            result = (result * base) % modulus
        exponent = exponent // 2
        base = (base * base) % modulus
    return result

In [720]:
def pickRandomPrime ():
    # get the random prime number between 128 bits and 225 bits
    smallValue = pow(2,127)
    largeValue = pow(2, 225 -1)
    while (True):
        prime = random.randint(smallValue, largeValue)
        if (fermatTest(prime)):
            return prime



In [721]:
# Fermat's primality test to check if a number is prime
def fermatTest (prime):
    if (prime < 2):
        return False
    ## check 2 time
    for i in range(2):
        a = random.randint(1, prime - 1)
        if (modularExponentiation(a, prime - 1, prime) != 1):
            return False
    return True

In [722]:
# extended euclidean algorithm use recursion to find the gcd
def extendedGCD(e, phi):
    # if e is 0 then phi is the gcd
    if (e == 0):
        return phi, 0, 1
                # recursively call extendedGCD unit the base case
    gcd, x1, y1 = extendedGCD(phi % e, e)
    x = y1 - (phi // e) * x1
    y = x1
    return gcd, x, y

In [723]:
# use the extended euclidean algorithm to find the modular inverse
def modularInverse(e, phi):
    #gets the gsd rom extendedGCD
    gcd, x, y = extendedGCD(e, phi)
    if (gcd != 1):
        print("ERROR: In modularInverse()")
    else:
        return x % phi


In [724]:
#generate the two keys
def RSA_key_generation():
    # prime numbers
    p = pickRandomPrime()
    q = pickRandomPrime()

    #n and phi calculation
    n = p * q
    phi = (p - 1) * (q - 1)

    #calculate e and keeps running until gcd =1
    while (True):
        e = random.randint(2, phi - 1)
        if (math.gcd(e, phi) == 1):
            break
    #calculate d 
    d = modularInverse(e, phi)

    # Open a file in write mode ('w') and save the integer
    with open('p_q.txt', 'w') as file:
        file.write(f"{p}\n")  # save p
        file.write(f"{q}")  # save p

    #write the public and private keys to a file
    with open("e_n.txt", "w") as f:
        f.write(f"{e}\n{n}")
    with open("d_n.txt", "w") as f:
        f.write(f"{d}\n{n}")

    print("done with key generation!")



In [725]:
# Read keys from file
def readKey(filename):
    with open(filename, "r") as f:
        # reads the line from the file and converts it to a int
        key = [int(line.strip()) for line in f.readlines()]
    return key[0], key[1]

In [726]:
# Signing function
def Signing(doc, key):
    # Read private key 
    d, n = readKey(key)
    
    with open(doc, "rb") as f:
        content = f.read()
    
    # Generate SHA-256 hash in bytes
    fileHash = hashlib.sha256(content).digest()

    # Sign the hash wiht key
    signature = modularExponentiation(int.from_bytes(fileHash), d, n)
    
    # Save new signed file
    with open(doc + ".signed", "wb") as f:
        f.write(content)
        f.write(signature.to_bytes((signature.bit_length() + 7) // 8, 'big'))
    
    print("\nSigned...")

In [727]:
# Verification function
def verification(doc, key):
    # Read public key
    e, n = readKey(key)
    
    with open(doc, "rb") as f:
        content = f.read()
    
    # Separate the original content and the signature
    signatureSize = (n.bit_length() + 7) // 8
    #  get original content the last 256 bits
    originalContent = content[:-signatureSize]
    # convert the signature to a int
    signature = int.from_bytes(content[-signatureSize:])
    
    # Generate SHA-256 hash of the original content
    newHashValue = hashlib.sha256(originalContent).digest()
    
    # Verify the signature with public key
    decryptedHash = modularExponentiation(signature, e, n).to_bytes(len(newHashValue))
    
    if (decryptedHash == newHashValue):
        print("\nAuthentic!")
    else:
        print("\nModified!")

In [728]:
# Main function to handle signing and verification
def CPSC_435_Project1(part, task="", fileName=""):
    if part == 1:
        RSA_key_generation()
    else:
        if "s" in task:  # Signing
            Signing(fileName, "d_n.txt")
        else:  # Verification
            verification(fileName, "e_n.txt")
    print("done!")


In [729]:
CPSC_435_Project1(1)

done with key generation!
done!


In [730]:
docName = "testfile.txt"
CPSC_435_Project1(2, "s", docName)


Signed...
done!


In [731]:
docName = "testfile.txt.signed"
CPSC_435_Project1(2, "v", docName)


Authentic!
done!
